In [ ]:
#All Libraries used in this project
import warnings
import datetime as dt
import numpy as np
import pandas as pd
from pandas_datareader import data as pdr
from tkinter import Tk
from tkinter.filedialog import askopenfilename
import matplotlib.pyplot as plt
%pip install pandas-ta
import pandas_ta as ta
import time
from warnings import filterwarnings
import numpy as np
import pandas as pd
from pandas_datareader import data as pdr
%pip install ta
from ta.trend import MACD, ADXIndicator
from ta.momentum import RSIIndicator
from ta.momentum import StochasticOscillator

# Market Data 
%pip install yahoo_fin
from yahoo_fin import stock_info as si
%pip install yfinance
import yfinance as yf

#Graphing/Visualization
import datetime as dt 
import plotly
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
warnings.filterwarnings("ignore")


#                                        ------- plotStock ---------                                                            #
#                                                                                                                               #
# A function used to plot the stocks with indicators. You can add more indicators to the charts via the different rows in the   #
# figure.                                                                                                                       #
#                                                                                                                               #
#-------------------------------------------------------------------------------------------------------------------------------#

#
def plotStock(df, macd, rsi, stock):
  # Declare plotly figure (go)
  fig=go.Figure()

  # add subplot properties when initializing fig variable
  fig = plotly.subplots.make_subplots(rows=6, cols=1, shared_xaxes=True,
                      vertical_spacing=0.01, 
                      row_heights=[0.5,0.1,0.2,0.2,0.2,0.2])

  fig.add_trace(go.Candlestick(x=df.index,
                  open=df['Open'],
                  high=df['High'],
                  low=df['Low'],
                  close=df['Close'], name = 'market data'))

  fig.add_trace(go.Scatter(x=df.index, 
                           y=df['MA5'], 
                           opacity=0.7, 
                           line=dict(color='blue', width=2), 
                           name='MA 5'))

  fig.add_trace(go.Scatter(x=df.index, 
                           y=df['MA20'], 
                           opacity=0.7, 
                           line=dict(color='orange', width=2), 
                           name='MA 20'))

  # Plot volume trace on 2nd row
  colors = ['green' if row['Open'] - row['Close'] >= 0 
            else 'red' for index, row in df.iterrows()]
  fig.add_trace(go.Bar(x=df.index, 
                       y=df['Volume'],
                       marker_color=colors
                      ), row=2, col=1)

  # Plot MACD trace on 3rd row
  colorsM = ['green' if val >= 0 
            else 'red' for val in macd.macd_diff()]
  fig.add_trace(go.Bar(x=df.index, 
                       y=macd.macd_diff(),
                       marker_color=colorsM
                      ), row=3, col=1)
  fig.add_trace(go.Scatter(x=df.index,
                           y=macd.macd(),
                           line=dict(color='black', width=2)
                          ), row=3, col=1)
  fig.add_trace(go.Scatter(x=df.index,
                           y=macd.macd_signal(),
                           line=dict(color='blue', width=1)
                          ), row=3, col=1)

  # update layout by changing the plot size, hiding legends & rangeslider, and removing gaps between dates
  fig.update_layout(height=900, width=1200, 
                    showlegend=False, 
                    xaxis_rangeslider_visible=False)


  # Make the title dynamic to reflect whichever stock we are analyzing
  fig.update_layout(
      title= str(stock)+' Live Share Price:',
      yaxis_title='Stock Price (USD per Shares)') 

  # update y-axis label
  fig.update_yaxes(title_text="Price", row=1, col=1)
  fig.update_yaxes(title_text="Volume", row=2, col=1)
  fig.update_yaxes(title_text="MACD", showgrid=False, row=3, col=1)
  fig.update_yaxes(title_text="Stoch", row=4, col=1)
  fig.update_yaxes(title_text="RSI", row=5, col=1)
  fig.update_yaxes(title_text="DMI", row=6, col=1)

  fig.update_xaxes(
      rangeslider_visible=False,
      rangeselector_visible=False,
      rangeselector=dict(
          buttons=list([
              dict(count=15, label="15m", step="minute", stepmode="backward"),
              dict(count=45, label="45m", step="minute", stepmode="backward"),
              dict(count=1, label="HTD", step="hour", stepmode="todate"),
              dict(count=1, label="1h", step="hour", stepmode="backward"),
              dict(count=4, label="4h", step="hour", stepmode="backward"),
              dict(count=1, label="1d", step="day", stepmode="backward"),
              dict(count=7, label="1wk", step="day", stepmode="backward"),
              dict(count=30, label="1mo", step="day", stepmode="backward"),
              dict(step="all")
          ])
      )
  )
  fig.show()
  time.sleep(2)
  plt.pause(0.005)

#                                        ------- scanStock ---------                                                            #
#                                                                                                                               #
# A function used to specify a different criteria for a pattern. In this case, we are only looking for undervalued stocks which #
# the MACD line has just crossed over the MACD signal line since the last unit of time. Each unit of time is specified in the   #
# checkStocks Function                                                                                                          #
#                                                                                                                               #
#-------------------------------------------------------------------------------------------------------------------------------#

def scanStock(df):
  score = 0
  undervalued = False
  macdCross = False
  if(len(df) >= 39):
    if (df['Close'][len(df)-1] is not None and df['bookValue'][len(df)-1] is not None):
      if (df['Close'][len(df)-1] < df['bookValue'][len(df)-1]):
        undervalued = True
        score+=1
        print("UNDERVALUED");
    if(df['MACD'][len(df)-1] > df['MACD_sig'][len(df)-1]):
      macdCross = True
      score+=1
      print("MACD CROSSED")
  return score, macdCross

#                                        ------- checkStocks ---------                                                           #
#                                                                                                                                #
# This function is used to retrieve the information from yFinance library as a DataFrame in pandas. In this function, we specify #
# the time frames in which we want to view the charts, in this case we use monthly candlestick charts with a 5 year overview.    #
# this function also calculates specified indicators and puts the info in the DataFrame so it can be accessed for plotting the   #
# charts.                                                                                                                        #
#--------------------------------------------------------------------------------------------------------------------------------#

def checkStocks(stockList):
  # Override Yahoo Finance
  yf.pdr_override()

  # Create input field for our desired stock 
  #stock=input("Enter a stock ticker symbol: ")

  for i in range(len(stockList)):
    stock = stockList[i]
    print(stock)

    # Retrieve stock data frame (df) from yfinance API at an interval of 1m 
    df = yf.download(tickers=stock,period='5y',interval='1mo')

    getInfo = yf.Ticker(stock)

    df['MA5'] = df['Close'].rolling(window=5).mean()
    df['MA20'] = df['Close'].rolling(window=20).mean()

    #Check for length of 39 so that the indicators have values via their windows.
    #Otherwise error when plotting the charts
    if(len(df) >= 39):
      #ADX
      adx = ADXIndicator(df["High"], df["Low"], df["Close"], window=14, fillna=True)

      #RSI
      rsi = RSIIndicator(close=df['Close'], window=14, fillna=True)

      # MACD
      macd = MACD(close=df['Close'], 
                  window_slow=26,
                  window_fast=12, 
                  window_sign=9)
      
      df['RSI'] = rsi.rsi()
      df['MACD'] = macd.macd()
      df['MACD_diff'] = macd.macd_diff()
      df['MACD_sig'] = macd.macd_signal()
      if('bookValue' in getInfo.info):
        df['bookValue'] = getInfo.info['bookValue']
      else:
        df['bookValue'] = 0
    else:
      df['bookValue'] = 0

    thisStockScore, dmiCross = scanStock(df)

    if(dmiCross):
      print("\n \033[1m" + str(stock) + " Book Value: ")
      print(df['bookValue'])
      #goodStocks.append(stock)
      plotStock(df, macd, rsi, stock)

In [ ]:
#This section retrieves all of the stock tickers from the specified exchanges and puts them all through the checkStocks Function

# gather stock symbols from major US exchanges
df1 = pd.DataFrame( si.tickers_sp500() )
df2 = pd.DataFrame( si.tickers_nasdaq() )
df3 = pd.DataFrame( si.tickers_dow() )
df4 = pd.DataFrame( si.tickers_other() )

df1.reset_index

# convert DataFrame to list, then to sets
sym1 = set( symbol for symbol in df1[0].values.tolist() )
sym2 = set( symbol for symbol in df2[0].values.tolist() )
sym3 = set( symbol for symbol in df3[0].values.tolist() )
sym4 = set( symbol for symbol in df4[0].values.tolist() )

# join the 4 sets into one. Because it's a set, there will be no duplicate symbols
symbols = set.union( sym1, sym2, sym3, sym4 )

# Some stocks are 5 characters. Those stocks with the suffixes listed below are not of interest.
my_list = ['W', 'R', 'P', 'Q']
del_set = set()
sav_set = set()

for symbol in symbols:
    if len( symbol ) > 4 and symbol[-1] in my_list:
        del_set.add( symbol )
    else:
        sav_set.add( symbol )

print( f'Removed {len( del_set )} unqualified stock symbols...' )
print( f'There are {len( sav_set )} qualified stock symbols...' )
#goodStocks = checkStocks(stockList)
presetList=[]
#checkStocks(presetList)

#testList = df1.drop()
#checkStocks(presetList)
checkStocks(df1[0])
checkStocks(df2[0])
checkStocks(df3[0])





In [ ]:
#print(df1[0])
#df1 = df1.drop([0])
print(df2[0][91])
#plotStock()